**Revised on 3/5/2024: Changed source files**

This is the skeleton code for Task 1 of the midterm project. The files that are downloaded in step 4 are based on the [Ember 2018 dataset](https://arxiv.org/abs/1804.04637), and contain the features (and corresponding labels) extracted from 1 million PE files, split into 80\% training and 20\% test datasets. The code used for for feature extraction is available [here](https://colab.research.google.com/drive/16q9bOlCmnTquPtVXVzxUj4ZY1ORp10R2?usp=sharing). However, the preprocessing and featurization process may take up to 3 hours on Google Colab. Hence, I recommend using the processed datasets (Step 4) to speed up your development.

Also, note that there is a new optional step 8.5 - To speed up your experiments, you may want to sample the original dataset of 800k training samples and 200k test samples to smaller datasets.

**Step 1:** Mount your Google Drive by clicking on "Mount Drive" in the Files section (panel to the left of this text.)

**Step 2:** Go to Runtime -> Change runtime type and select T4 GPU.

**Step 3:** Create a folder in your Google Drive, and rename it to "vMalConv"

**Step 4:** Download the pre-processed training and test datasets.

In [ ]:
# ~8GB
!wget https://dsci6015s24-midterm.s3.amazonaws.com/v2/X_train.dat
!wget https://dsci6015s24-midterm.s3.amazonaws.com/v2/X_test.dat
!wget https://dsci6015s24-midterm.s3.amazonaws.com/v2/y_train.dat
!wget https://dsci6015s24-midterm.s3.amazonaws.com/v2/y_test.dat
!wget https://dsci6015s24-midterm.s3.amazonaws.com/v2/metadata.csv

--2024-03-14 12:18:40--  https://dsci6015s24-midterm.s3.amazonaws.com/v2/X_train.dat
Resolving dsci6015s24-midterm.s3.amazonaws.com (dsci6015s24-midterm.s3.amazonaws.com)... 52.216.35.169, 54.231.201.249, 52.217.226.249, ...
Connecting to dsci6015s24-midterm.s3.amazonaws.com (dsci6015s24-midterm.s3.amazonaws.com)|52.216.35.169|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7619200000 (7.1G) [binary/octet-stream]
Saving to: ‘X_train.dat’

X_train.dat         100%[===================>]   7.10G  43.9MB/s    in 2m 41s  

2024-03-14 12:21:22 (45.2 MB/s) - ‘X_train.dat’ saved [7619200000/7619200000]

--2024-03-14 12:21:22--  https://dsci6015s24-midterm.s3.amazonaws.com/v2/X_test.dat
Resolving dsci6015s24-midterm.s3.amazonaws.com (dsci6015s24-midterm.s3.amazonaws.com)... 54.231.172.233, 54.231.171.201, 52.217.196.161, ...
Connecting to dsci6015s24-midterm.s3.amazonaws.com (dsci6015s24-midterm.s3.amazonaws.com)|54.231.172.233|:443... connected.
HTTP request sent, awa

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Step 5:** Copy the downloaded files to vMalConv

In [ ]:
!cp /content/X_train.dat /content/drive/MyDrive/vMalConv/X_train.dat
!cp /content/X_test.dat /content/drive/MyDrive/vMalConv/X_test.dat
!cp /content/y_train.dat /content/drive/MyDrive/vMalConv/y_train.dat
!cp /content/y_test.dat /content/drive/MyDrive/vMalConv/y_test.dat
!cp /content/metadata.csv /content/drive/MyDrive/vMalConv/metadata.csv

**Step 6:** Download and install Ember:

In [ ]:
!pip install git+https://github.com/PFGimenez/ember.git

  Cloning https://github.com/PFGimenez/ember.git to /tmp/pip-req-build-9hg8qa5y
  Running command git clone --filter=blob:none --quiet https://github.com/PFGimenez/ember.git /tmp/pip-req-build-9hg8qa5y
  Resolved https://github.com/PFGimenez/ember.git to commit 3b82fe63069884882e743af725d29cc2a67859f1
  Preparing metadata (setup.py) ... done
  Created wheel for ember: filename=ember-0.1.0-py3-none-any.whl size=13050 sha256=687f55561796a26e201358442062921d497e3b2b1a057dadc7d9861317c87907
  Stored in directory: /tmp/pip-ephem-wheel-cache-u7dfkzmq/wheels/8f/69/f9/1917c8df03b25fe53e8e2f6cb2c9f61a43dec179b19b10ab9f
Successfully built ember


In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
import ember
import altair as alt

In [ ]:
!pip install lief

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 9.6 MB/s eta 0:00:00


**Step 7:** Read vectorized features from the data files.

In [ ]:
import ember
X_train, y_train, X_test, y_test = ember.read_vectorized_features("drive/MyDrive/vMalConv/")
metadata_dataframe = ember.read_metadata("drive/MyDrive/vMalConv/")

**Step 8:** Get rid of rows with no labels.

In [ ]:
 labelrows = (y_train != -1)
 X_train = X_train[labelrows]
 y_train = y_train[labelrows]

In [ ]:
import h5py
h5f = h5py.File('X_train.h5', 'w')
h5f.create_dataset('X_train', data=X_train)
h5f.close()
h5f = h5py.File('y_train.h5', 'w')
h5f.create_dataset('y_train', data=y_train)
h5f.close()

In [ ]:
!cp /content/X_train.h5 /content/drive/MyDrive/vMalConv/X_train.h5
!cp /content/y_train.h5 /content/drive/MyDrive/vMalConv/y_train.h5

**Optional Step 8.5:** To speed up your experiments, you may want to sample the original dataset of 800k training samples and 200k test samples to smaller datasets. You can use the [Pandas Dataframe sample() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html), or come up with your own sampling methodology. Be mindful of the fact that the database is heavily imbalanced.

In [ ]:
import numpy as np

samples_per_class = 100000

unique_labels = np.unique(y_train[y_train!=-1])

selected_indices = []

for label in unique_labels:
    label_indices = np.where(y_train == label)[0]
    random_indices = np.random.choice(label_indices, samples_per_class, replace=True)
    selected_indices.extend(random_indices.tolist())

X_train = X_train[selected_indices]
y_train = y_train[selected_indices]


print(X_train.shape)
print(y_train.shape)

(200000, 2381)
(200000,)


> **Task 1:** Complete the following code to build the architecture of MalConv in PyTorch:

In [ ]:
import torch
import torch.nn as nn

class MalConv(nn.Module):
    def __init__(self, input_length=2000000, embedding_dim=8, window_size=8, output_dim=1):
        super(MalConv, self).__init__()
        self.embedding_dim = embedding_dim
        self.window_size = window_size
        self.input_length = input_length
        self.flatten = nn.Flatten()

        self.embed = nn.Embedding(256, embedding_dim)  # 256 unique bytes, embedding dimension
        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=128, kernel_size=window_size, stride=window_size, bias=True)
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=128, kernel_size=window_size, stride=window_size, bias=True)
        conv_output_length = self.calculate_conv_output_length()

        self.fc1 = nn.Linear(conv_output_length, 128)
        self.fc2 = nn.Linear(128, output_dim)
        self.sigmoid = nn.Sigmoid()

    def calculate_conv_output_length(self):
        conv1_output_length = (self.input_length - self.window_size) // self.window_size + 1
        conv2_output_length = (conv1_output_length - self.window_size) // self.window_size + 1
        conv_output_length = 128 * conv2_output_length
        return conv_output_length

    def forward(self, x):
        x = self.embed(x)
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x


**Step 8:** Partial fit the standardScaler to avoid overloading the memory:

In [ ]:
from sklearn.preprocessing import StandardScaler
mms = StandardScaler()
for x in range(0,100000,1000):
  mms.partial_fit(X_train[x:x+1000])

In [ ]:
X_train = mms.transform(X_train)
X_train = np.array((X_train+1)*127.5, dtype=np.uint8)

In [ ]:
## Reshape to create 3 channels ##
import numpy as np
X_train = np.reshape(X_train,(-1,2381))
y_train = np.reshape(y_train,(-1,1))

In [ ]:
y_train.shape

(200000, 1)

In [ ]:
X_train.shape

(200000, 2381)

**Load, Tensorize, and Split** The following code takes care of converting the training data into Torch Tensors, and then splits it into 80% training and 20% validation datasets.

In [ ]:
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split




X_train = torch.tensor(X_train, dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.float32)


X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)


train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

batch_size = 2048
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

> **Task 2:** Complete the following code to train the model on the GPU for 15 epochs, with a batch size of 64. If you are on Google Colab, don't forget to change the kernel in Runtime -> Change runtime type -> T4 GPU.

In [ ]:
# Initialize the MalConv model
model = MalConv(input_length=2381)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MalConv(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (embed): Embedding(256, 8)
  (conv1): Conv1d(8, 128, kernel_size=(8,), stride=(8,))
  (conv2): Conv1d(128, 128, kernel_size=(8,), stride=(8,))
  (fc1): Linear(in_features=4736, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [ ]:
import os


criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

save_dir = "drive/MyDrive/vMalConv/"


num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Training Loss: {running_loss/len(train_loader)}')


    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    print(f'Validation Loss: {val_loss/len(val_loader)}')


    if (epoch + 1) % 5 == 0:
        checkpoint_path = os.path.join(save_dir, f'model_epoch_{epoch+1}.pt')
        torch.save(model.state_dict(), checkpoint_path)
        print(f'Model checkpoint saved to {checkpoint_path}')


Epoch 1, Training Loss: 0.5192282098003581
Validation Loss: 0.36930969208478925
Epoch 2, Training Loss: 0.31661013485510137
Validation Loss: 0.3332425907254219
Epoch 3, Training Loss: 0.26490598954731903
Validation Loss: 0.27115564197301867
Epoch 4, Training Loss: 0.2381400320726105
Validation Loss: 0.2666808933019638
Epoch 5, Training Loss: 0.2207106612151182
Validation Loss: 0.2322502538561821
Model checkpoint saved to drive/MyDrive/vMalConv/model_epoch_5.pt
Epoch 6, Training Loss: 0.21052848179883596
Validation Loss: 0.2419934332370758
Epoch 7, Training Loss: 0.1974210810812214
Validation Loss: 0.20599483251571654
Epoch 8, Training Loss: 0.1860680084062528
Validation Loss: 0.220376905053854
Epoch 9, Training Loss: 0.18230176009709323
Validation Loss: 0.19731391966342926
Epoch 10, Training Loss: 0.1748107490660269
Validation Loss: 0.24148470163345337
Model checkpoint saved to drive/MyDrive/vMalConv/model_epoch_10.pt


**Task 3:** Complete the following code to evaluate your trained model on the test data.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score


X_test = mms.transform(X_test[:50000])
y_test = y_test[:50000]
X_test = np.array((X_test+1)*127.5, dtype=np.uint8)

X_test = torch.tensor(X_test, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.float32)


test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()

<ipython-input-26-09661a2b7984>:6: RuntimeWarning: invalid value encountered in cast
  X_test = np.array((X_test+1)*127.5, dtype=np.uint8)


MalConv(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (embed): Embedding(256, 8)
  (conv1): Conv1d(8, 128, kernel_size=(8,), stride=(8,))
  (conv2): Conv1d(128, 128, kernel_size=(8,), stride=(8,))
  (fc1): Linear(in_features=4736, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [ ]:
predictions = []
labels = []

with torch.no_grad():
    for inputs, labels_batch in test_loader:

        inputs, labels_batch = inputs.to(device), labels_batch.to(device)
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()

        predictions.extend(predicted.cpu().numpy())
        labels.extend(labels_batch.cpu().numpy())


accuracy = accuracy_score(labels, predictions)
precision = precision_score(labels, predictions)
recall = recall_score(labels, predictions)

print(f'Test Accuracy: {accuracy:.5f}')
print(f'Precision: {precision:.5f}')
print(f'Recall: {recall:.5f}')

Test Accuracy: 0.49888
Precision: 0.37500
Recall: 0.00036


**Task 4:** Comment on the results in this text box.

The  model achieved a test accuracy of 49.89%, indicating a slight improvement over random guessing. However, with a precision of 37.50% and a recall of 0.036%, it struggles to effectively differentiate between classes, exhibiting a significant presence of false positives and false negatives. To enhance its performance, consider refining the model architecture, augmenting the dataset, or optimizing hyperparameters. These adjustments may improve the model's precision, recall, and overall predictive accuracy, making it more reliable in practical scenarios.